## Hierarchically consistent forecasting with EXPERT

1. We forecast on multiple hierarchical levels with a dataset containing revenues. We will forecast on the country level, the region level and finally one time series which represents the global aggregate.
2. After forecasting the time series individually on the three different hierarchical levels, we will reconcile the forecasts to achieve hierarchically consistent forecasts.


For more information about this dataset see the [sales forecasting use case](../use_cases/sales_forecasting/sales_forecasting.ipynb).

In [1]:
from futureexpert import DataDefinition, ExpertClient, FileSpecification, TsCreationConfig
import futureexpert.checkin as checkin
import time

client = ExpertClient()

INFO:futureexpert.expert_client:Successfully logged in for group group-expert.


## CHECK-IN configuration
In this step, we transform our CSV data into structured time series. First, we define the data structure. For each column, we specify whether it contains the date, grouping information (in our case hierarchical information about the sales regions), or value data. 
During the time series creation process, we choose to aggregate the data at the “Region” and “Country” level. We also enable the `save_hierarchy` so the information about the hierarchical structure persists. 

For more informationa bout the CHECK-IN Process see the [checkin configuration options](./checkin_configuration_options.ipynb).

In [2]:
# check in consumptions
actuals_version_id = client.check_in_time_series(raw_data_source='../use_cases/sales_forecasting/demo_sales_data.csv',
                                                  data_definition=DataDefinition(date_column=checkin.DateColumn(name='Date', format='%d.%m.%Y'),
                                                                                 value_columns=[checkin.ValueColumn(name='Sales')],
                                                                                 group_columns=[checkin.GroupColumn(name='Region'),
                                                                                                checkin.GroupColumn(name='Country')],
                                                                                 remove_columns=[3],),
                                                  config_ts_creation=TsCreationConfig(time_granularity='monthly',
                                                                                      start_date="2016-03-01",
                                                                                      value_columns_to_save=['Sales'],
                                                                                      grouping_level=['Region', 'Country'],
                                                                                      save_hierarchy=True,
                                                                                      filter=[checkin.FilterSettings(type='exclusion', variable='Region', items=['Global']),
                                                                                              checkin.FilterSettings(type='exclusion', variable='Country', items=['Global'])],
                                                                                      missing_value_handler="keepNaN",),
                                                  file_specification=FileSpecification(delimiter=';', decimal='.'))

INFO:futureexpert.expert_client:Transforming input data...
INFO:futureexpert.expert_client:Creating time series using CHECK-IN...
INFO:futureexpert.expert_client:Finished time series creation.


## Forecasting

With CHECK-IN having prepared our time series, we are ready to configure the FORECAST settings:
- We define a set of forecasting methods with fast performances. For more advanced settings and explanations see the [sales forecasting use case](../use_cases/sales_forecasting/sales_forecasting.ipynb) 

In [18]:
from futureexpert import ForecastingConfig, MethodSelectionConfig, PreprocessingConfig, ReportConfig
fc_methods = ['LinearRegression','Naive', 'MA(granularity)','MA(3)', 'MA(season lag)','FoundationModel']
fc_report_config = ReportConfig(title='Monthly Sales Forecast on Multiple Hierarchical Levels',
                                preprocessing=PreprocessingConfig(detect_outliers=True,
                                                                  replace_outliers=True,
                                                                  detect_changepoints=True),
                                forecasting=ForecastingConfig(fc_horizon=12,
                                                              lower_bound=0,
                                                              use_ensemble=False,
                                                              confidence_level=0.90),
                                method_selection=MethodSelectionConfig(number_iterations=12,
                                                                       refit=True,
                                                                       step_weights={1:1., 2:1., 3:1., 4:0.5, 5:0.5, 6:0.5},
                                                                       forecasting_methods=fc_methods),
                                max_ts_len=72)

forecast_identifier = client.start_forecast(version=actuals_version_id, config=fc_report_config)
print(forecast_identifier)

INFO:futureexpert.expert_client:Preparing data for forecast...
INFO:futureexpert.expert_client:Finished data preparation for forecast.
INFO:futureexpert.expert_client:Started creating forecasting report with FORECAST...
INFO:futureexpert.expert_client:Report created with ID 131221. Forecasts are running...


report_id=131221 settings_id=131246


In [ ]:

# Watch the current status of the forecasting report
while not (current_status := client.get_report_status(id=forecast_identifier)).is_finished:
    time.sleep(20)  # Wait between status requests
# Retrieve the results of the forecasting report
current_status.print()

Status forecasting report for id: report_id=131221 settings_id=131246
 100 % are finished 
 10 time series requested for calculation 
 10 time series finished 
 0 time series without evaluation 
 0 time series ran into an error


In [22]:
results = client.get_fc_results(id=forecast_identifier , include_backtesting=True, include_k_best_models=3)
results.sort(key=lambda x: x.input.actuals.name)

### Forecast results

The forecasts were created seperately for each time series. Because they were created independently the sum of the hierarchical levels (Global, Region and Country) can be slightly inconsistent. This ambiguity hinders planing processes.

In [ ]:
from futureexpert.forecast import export_forecasts_to_pandas
forecasts_df = export_forecasts_to_pandas(results)
forecasts_df_june = forecasts_df[forecasts_df.time_stamp_utc == forecasts_df.time_stamp_utc.iloc[0]]

In [ ]:
global_sales = forecasts_df_june[forecasts_df_june.name == 'Sales'].point_forecast_value.iloc[0]
europe_sales = forecasts_df_june[forecasts_df_june.name == 'Sales-Europe'].point_forecast_value.iloc[0]
na_sales = forecasts_df_june[forecasts_df_june.name == 'Sales-North America'].point_forecast_value.iloc[0]
if global_sales != europe_sales + na_sales:
    print('Sales on region level is incosistent to global level!')

## Make forecasts consistent

The process of making time series hierarchically consistent is called hierarchical reconciliation.
- We need to specify which actuals and which hierarchically inconsistent forecast is the basis of the reconciliation.
- Then we need to specify the specific reconciliation method.

In [ ]:
from futureexpert import MakeForecastConsistentConfiguration, ReconciliationConfig, MakeForecastConsistentDataSelection, ReconciliationMethod
from futureexpert.make_forecast_consistent import export_consistent_forecasts_to_pandas

In [ ]:
data_selection = MakeForecastConsistentDataSelection(
    version=actuals_version_id,
    fc_report_id=forecast_identifier.report_id
)
hcfc_config = ReconciliationConfig(
    method=ReconciliationMethod.MIN_TRACE_WLS_STRUCT,
    fallback_methods=[],
)
hcfc_run_config = MakeForecastConsistentConfiguration(
    data_selection=data_selection,
    report_note="My hierarchical forecasting report",
    reconciliation=hcfc_config
)

In [ ]:
report_id = client.start_making_forecast_consistent(hcfc_run_config)
hcfc_results = client.get_consistent_forecast_results(report_id)

INFO:futureexpert.expert_client:Preparing data for forecast consistency...
INFO:futureexpert.expert_client:Finished data preparation for forecast consistency.
INFO:futureexpert.expert_client:Started creating hierarchical reconciliation for consistent forecasts...
INFO:futureexpert.expert_client:Report created with ID 131228. Reconciliation is running...


ReportIdentifier(report_id=131228, settings_id=131253)

In [58]:
consistent_fc_df = export_consistent_forecasts_to_pandas(hcfc_results)
consistent_fc_df_june = consistent_fc_df[consistent_fc_df.time_stamp_utc == consistent_fc_df.time_stamp_utc.iloc[0]]

In [60]:
global_sales = consistent_fc_df_june[consistent_fc_df_june.name == 'Sales'].point_forecast_value.iloc[0]
europe_sales = consistent_fc_df_june[consistent_fc_df_june.name == 'Sales-Europe'].point_forecast_value.iloc[0]
na_sales = consistent_fc_df_june[consistent_fc_df_june.name == 'Sales-North America'].point_forecast_value.iloc[0]
assert global_sales == europe_sales + na_sales, ('Sales on region level is incosistent to global level!')
print('Sucessfully created consistent forecasts!')

Sucessfully created consistent forecasts!
